In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_submession=pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
df_submession.shape


In [ ]:
df_train=pd.read_csv ("/kaggle/input/llm-classification-finetuning/train.csv") 
df_train.columns

In [ ]:
df_train.shape

In [ ]:
df_test=pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
df_test.columns 

In [ ]:
df_test.describe()

In [ ]:
 df_train.head(3) 


In [ ]:
df_test.head()

In [ ]:
df_train.columns


In [ ]:
# Concatenar os textos
df_train['combined_text'] = df_train['prompt'] + " [SEP] " + df_train['response_a'] + " [SEP] " + df_train['response_b']

# Preparar o dataset de teste
df_test['combined_text'] = df_test['prompt'] + " [SEP] " + df_test['response_a'] + " [SEP] " + df_test['response_b']

# Concatenar as colunas
df_train['label'] = df_train[['winner_model_a', 
                              'winner_model_b', 
                              'winner_tie']].idxmax(axis=1)

# Visualizando
df_train.head(n=10)


In [ ]:
# Mapear labels para inteiros
label_mapping = {'winner_model_a': 0, 
                 'winner_model_b': 1, 
                 'winner_tie': 2}

# Variável alvo
df_train['label'] = df_train['label'].map(label_mapping)
df_train.head(3)

In [ ]:
df_train.tail(3)

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.shape

In [ ]:
# Importing necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns

# View missing values with a different color palette
plt.figure(figsize=(10, 6))
sns.heatmap(df_train.isnull(), cbar=False, cmap="coolwarm")  # Changed color palette to 'coolwarm'
plt.title("Viewing Missing Values in the Training Set")
plt.show()


In [ ]:
df_train['winner_model_a'].value_counts()


In [ ]:
# Importing necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Distribution of the Target Variable "Class" with changed color
sns.countplot(x='label', data=df_train, palette='coolwarm')  # Changed color palette to 'coolwarm'
plt.title("Class Distribution of Binding Free Energy (BFE)")
plt.grid(False)
plt.show()

print("\nClass Distribution:")
print(df_train['label'].value_counts())


In [ ]:
# Importing necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Distribution of the Target Variable "Class" with normal curve
plt.figure(figsize=(10, 6))
sns.countplot(x='label', data=df_train, palette='coolwarm', alpha=0.7)  # Countplot with color
sns.kdeplot(df_train['label'], color='black', lw=2)  # Adding KDE (normal curve) over the countplot
plt.title("Class Distribution of Binding Free Energy (BFE) with Normal Curve")
plt.grid(False)
plt.show()

print("\nClass Distribution:")
print(df_train['label'].value_counts())


In [ ]:
df_train.describe().T

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create a list of unique classes in the 'label' column
labels = df_train['label'].unique()

# Loop through each class and generate the word cloud
for label in labels:
    texts_prompt = ' '.join(df_train[df_train['label'] == label]['prompt'])

    # Generate the word cloud for each class
    wordcloud = WordCloud(width=800, height=400, 
                          background_color='white').generate(texts_prompt)

    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Word Cloud for Class {label}')
    plt.show()


In [ ]:
# Import necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Initialize TfidfVectorizers for each text column
vectorizer_prompt = TfidfVectorizer(max_features=500)   
vectorizer_resp_a = TfidfVectorizer(max_features=500)
vectorizer_resp_b = TfidfVectorizer(max_features=500)

# Fit and transform each text column for the training set
train_prompt_features = vectorizer_prompt.fit_transform(df_train['prompt'])
train_resp_a_features = vectorizer_resp_a.fit_transform(df_train['response_a'])
train_resp_b_features = vectorizer_resp_b.fit_transform(df_train['response_b'])

# Combine the TF-IDF matrices for the training set
train_combined_features = hstack([train_prompt_features, train_resp_a_features, train_resp_b_features])

# Fit and transform each text column for the test set
test_prompt_features = vectorizer_prompt.transform(df_test['prompt'])
test_resp_a_features = vectorizer_resp_a.transform(df_test['response_a'])
test_resp_b_features = vectorizer_resp_b.transform(df_test['response_b'])

# Combine the TF-IDF matrices for the test set
test_combined_features = hstack([test_prompt_features, test_resp_a_features, test_resp_b_features])

# Store the result for the test set
final_test_features = test_combined_features

# Feature matrix for the training set
final_train_features = train_combined_features


In [ ]:
X = test_combined_features
y = df_train [['winner_model_a','winner_model_b','winner_tie']]
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, df_train['label'], 
                                                  test_size=0.2, random_state=42, 
                                                  stratify=df_train['label'])

# Viewing training data
print("Viewing rows and columns of X_train:", X_train.shape)

# Viewing target data for training
print("Viewing rows and columns of y_train:", y_train.shape)
